In [ ]:
import torch
import warnings
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from zipfile import ZipFile
from PIL import Image
import yaml
from pathlib import Path


with open("cfg.yaml", "r") as file:
        cfg = yaml.safe_load(file)

sys.path.append("src/")


torch.manual_seed(1)
warnings.filterwarnings("ignore")

print(f"cuda is available: {torch.cuda.is_available()}")
print(f"mps is available: {torch.backends.mps.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available()
                else "mps" if torch.backends.mps.is_built() else "cpu")

import colorization_dataset
from net import VGG_Encoder, Decoder, Net, Net_with_LabVGG
from train_and_test import train, test_model
# from inference_generation import test_transform, style_transfer


### Testing Dataloader and color spaces

In [ ]:
train_data, test_data = colorization_dataset.prepare_dataset(train_size=cfg['train_size'], test_size=cfg['test_size'])
train_loader, _ = colorization_dataset.prepare_dataloader(train_data, test_data, batch_size=cfg['batch_size'])

In [ ]:
train_image_lab= next(iter(train_loader))["image"][5]

In [ ]:
print(train_image_lab[0].min(), train_image_lab[0].max())
print(train_image_lab[1].min(), train_image_lab[1].max())
print(train_image_lab[2].min(), train_image_lab[2].max())

In [ ]:
print(train_image_lab.permute(1,2,0).shape)

In [ ]:
plt.imshow(train_image_lab.permute(1,2,0).numpy())
plt.axis('off')
plt.show()

In [ ]:
labtorgb= colorization_dataset.LABtoRGB()
train_image_rgb= labtorgb(train_image_lab)

In [ ]:
print(train_image_rgb[0].min(), train_image_rgb[0].max())
print(train_image_rgb[1].min(), train_image_rgb[1].max())
print(train_image_rgb[2].min(), train_image_rgb[2].max())

print(train_image_rgb.shape)

In [ ]:
plt.imshow(train_image_rgb.permute(1,2,0).numpy())
plt.axis('off')
plt.show()

### Testing Network with CIE Lab VGG16

In [3]:
encoder = VGG_Encoder()
decoder = Decoder()
net= Net_with_LabVGG(encoder, decoder)
#net.encoder.load_weights("./model/LabVGG16_dict.pth")
net.encoder.load_state_dict(torch.load("./model/model_storage/encoder_sigmoid_iter_10.pth.tar"))
net.decoder.load_state_dict(torch.load("./model/model_storage/decoder_sigmoid_iter_10.pth.tar"))

<All keys matched successfully>

In [4]:
train(net,cfg)

Using device: mps

'All tests passed'

Dataset loaded successfully
Data loader prepared successfully


  0%|          | 0/100 [00:00<?, ?it/s]

  batch 10 loss: 0.017226193007081746


...

  batch 20 loss: 0.015184355527162552


reproduced_image min: 0.014796859584748745 max: 0.9647319912910461

  batch 30 loss: 0.014790221489965916


reproduced_image min: 0.01307931262999773 max: 0.9708022475242615

  4%|▍         | 4/100 [03:51<1:32:21, 57.72s/it]

In [ ]:
train_data, test_data = colorization_dataset.prepare_dataset(train_size=cfg['train_size'], test_size=cfg['test_size'])
train_loader, _ = colorization_dataset.prepare_dataloader(train_data, test_data, batch_size=cfg['batch_size'])
input= next(iter(train_loader))["image"][5]
plt.imshow(input.permute(1,2,0).numpy())
plt.axis('off')
plt.show()

In [ ]:
net=net.to(device)
if len(input.shape)==3:
    input=input.unsqueeze(0)
input=input.to(device)
print(input.shape)
encoded= net.encoder(input)
print(encoded.shape)
decoded=decoder(encoded)
print(decoded.shape)

decoded=decoded.cpu()
# plt.imshow(decoded.detach().numpy().squeeze().permute(1,2,0).numpy())
plt.imshow(decoded.squeeze(0).permute(1,2,0).detach().numpy())
plt.axis('off')
plt.show()

### Testing load of encoder weights back in the network (NOT UP TO DATE)

In [ ]:
encoder = LabVGG16()
decoder = Decoder()
#decoder.load_state_dict(torch.load(cfg["decoder"]))
net= Net(encoder, decoder)
net.enc_1.load_state_dict(torch.load(cfg["enc_1"]))
net.enc_2.load_state_dict(torch.load(cfg["enc_2"]))
net.enc_3.load_state_dict(torch.load(cfg["enc_3"]))
net.enc_4.load_state_dict(torch.load(cfg["enc_4"]))
net.decoder.load_state_dict(torch.load(cfg["decoder"]))

test_model(net, cfg)

### Inference test on AdaIN (NOT UP TO DATE)

In [ ]:
# This are the image and the style I want to mix
# --> Set their path in cfg.yaml
input_img=Image.open("data/content_dir/brad_pitt.jpg")
display(input_img)
style_ref=Image.open("data/style_dir/brushstrokes.jpg")
display(style_ref)

device = torch.device("cuda" if torch.cuda.is_available()
                else "mps" if torch.backends.mps.is_built() else "cpu")

output_dir = Path(cfg["output_dir"])
output_dir.mkdir(exist_ok=True, parents=True)

# Either --content or --contentDir should be given.
assert (cfg["content"] or cfg["content_dir"])
if cfg["content"]:
    content_paths = [Path(cfg["content"])]
else:
    content_dir = Path(cfg["content_dir"])
    content_paths = [f for f in content_dir.glob('*')]

# Either --style or --styleDir should be given.
assert (cfg["style"] or cfg["style_dir"])
if cfg["style"]:
    style_paths = cfg["style"].split(',')
    if len(style_paths) == 1:
        style_paths = [Path(cfg["style"])]
    else:
        do_interpolation = True
        assert (cfg["style_interpolation_weights"] != ''), \
            'Please specify interpolation weights'
        weights = [int(i) for i in cfg["style_interpolation_weights"].split(',')]
        interpolation_weights = [w / sum(weights) for w in weights]
else:
    style_dir = Path(cfg["style_dir"])
    style_paths = [f for f in style_dir.glob('*')]

encoder = VGG_Encoder()
decoder = Decoder()

decoder.eval()
encoder.eval()

decoder.load_state_dict(torch.load(cfg["decoder"]))
# encoder.load_state_dict(torch.load(args.vgg))
#vgg = nn.Sequential(*list(vgg.children())[:31])

encoder.to(device)
decoder.to(device)

content_tf = test_transform(cfg["content_size"], cfg["crop"])
style_tf = test_transform(cfg["style_size"], cfg["crop"])

do_interpolation = False

for content_path in content_paths:

    # |||||||
    # not in use
    # |||||||
    # vvvvvvv
    if do_interpolation:  # one content image, N style image
        style = torch.stack([style_tf(Image.open(str(p))) for p in style_paths])
        content = content_tf(Image.open(str(content_path))) \
            .unsqueeze(0).expand_as(style)
        style = style.to(device)
        content = content.to(device)
        with torch.no_grad():
            output = style_transfer(encoder, decoder, content, style,
                                    cfg["alpha"], interpolation_weights)
        output = output.cpu()
        output_name = output_dir / '{:s}_interpolation{:s}'.format(
            content_path.stem, cfg["save_ext"])
        torchvision.utils.save_image(output, str(output_name))
    # ^^^^^^^
    # |||||||
    # not in use
    # |||||||
        

    else:  # process one content and one style
        for style_path in style_paths:
            content = content_tf(Image.open(str(content_path)))
            style = style_tf(Image.open(str(style_path)))
            #if cfg["preserve_color"]:
            #    style = coral(style, content)
            style = style.to(device).unsqueeze(0)
            content = content.to(device).unsqueeze(0)
            with torch.no_grad():
                output = style_transfer(encoder, decoder, content, style,
                                        cfg["alpha"])
            output = output.cpu()
            output_name = output_dir / '{:s}_stylized_{:s}{:s}'.format(
                content_path.stem, style_path.stem, cfg["save_ext"])
            torchvision.utils.save_image(output, str(output_name))

output_image=Image.open("data/output_dir/brad_pitt_stylized_brushstrokes.jpg")
display(output_image)
